<a href="https://colab.research.google.com/github/SoumyaShreeram/Microlensing_with_NeuralNets/blob/master/04_Neural_Network_Designs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setting up the architectures for the Neural Networks

The following script defines 4 different NNs:
1. Custom NN
2. 

In [1]:
from google.colab import drive
import os
import pickle

import numpy as np
import matplotlib.pyplot as plt
import random
from IPython.display import Image

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

%tensorflow_version 2.x
import tensorflow as tf
import keras
from keras.models import Sequential, load_model
from keras.layers import (Dense, Conv1D, MaxPooling1D,
                          Dropout, Flatten, BatchNormalization)
from keras import optimizers
from keras import regularizers

TensorFlow 2.x selected.


Using TensorFlow backend.


In [0]:
def defineFNN1(num_pix):
   """
  Function to build the model architecture, set optimizers and compile the model 
  @num_pix :: used to define the shape of the input layer
  """
  model = Sequential()
  # 1 input layer, 1 hidden layer, and 1 output layer
  model.add(Dense(90, activation='relu', input_shape=(num_pix, None, 1)))
  model.add(Dense(30, activation='relu'))
  model.add(Dense(3, activation='softmax'))

  # optimizer 
  sgd = optimizers.SGD(lr=learning_rate, momentum=momentum, nesterov=False)

  # compilation for a multi-category classification problem
  model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
  return model